# Welcome to the Versatile Data Kit Demo Example!

## Workshop Steps
Now that you have opened up the MyBinder environment and are reading this, you are already on the right track! Inside this environment,
you will also find:
* samples: This is a folder containing the base of the scripts that you will be working with to finish the exercise. Please look for the triple exclamation points (!!!) as that means that you are being asked to write some code to get things to work!
* README.md: This is just the README file you saw on the Github page.
* requirements.txt: This is a list of the required libraries that were installed upon startup.
* exercise.ipynb: The file you are reading right now! Think of this as your home page.
* Other system files - postBuild and start: No need to worry about these. They are needed for the setup.

### Step 0: Explore VDK's Functionalities
A simple command like that found in the exercise.ipynb "!vdk --help" gives you all the information you need.


In [ ]:
!vdk --help

### Step 1: Identify the business process

**Business request**: 

We work for Volkswagen. Latest market research has shown that our customers need to understand better their battery usage in order to be able to plan their trips better. We want to create application that will allow them to predict battery drainage based on different environment parameters (speed of travel, are they using heated seats, etc.). Using the app our customers would be able to plan their trips much better (e.g they'd drive slowly or they'd not use the heated seats, etc.)



**What is the business process we want to model ?**

<font color='red'>**ATTENTION!**</font> ...

How do we find the business process 

* Look from the customer or user persona's point of view.
* It is usally expressed as action verbs/events. 
* It generates key performance indicators
* Often it might be supported by an operational system
* Communication: talk to product team, marketing team, customer excelence team and the customers themselves (if feasible).


**Task: Describe the business process for the business reuqest we are tracking. 
        In our case we will make it up. In reality *never* get tempted to make things up. It must be a real business process.**


<!-- The business process we need to model is a car owner taking a trip using VW electric cars we'll track the EV consumption.. -->

#### Ingest data

We are using the data provided by Göktuğ Özgül on Kaggle.com
Now let's ingest our data.  

We are going to an easy way to ingest CSV data using Versatile Data Kit: `vdk ingest-csv`. <br>
Make sure to check the help! It has pretty good documentation and examples. 
Full tutorial can also been seen [here](https://github.com/vmware/versatile-data-kit/wiki/Ingesting-local-CSV-file-into-Database).

In [ ]:
! vdk ingest-csv --help


The command did not work :) . That's because we need to install a plugin. Versatile Data Kit is extremely pluggbale and versatile. 

Let's install vdk-csv plugin: 

In [ ]:
!pip install vdk-csv

For this excercise we are going to use Trino. Let's tell VDK that we want to ingest and query data using Trino (the connection settingis preconfigured for us.). 

<font color='red'>**ATTENTION!**</font> ...

Change VDK_TRINO_SCHEMA with one of your own for example "aivanov" (for Antoni Ivanov) 

In [ ]:

%env VDK_DB_DEFAULT_TYPE=trino
%env VDK_INGEST_METHOD_DEFAULT=trino
%env VDK_TRINO_SCHEMA=ai4




In [ ]:
!vdk trino-query -q "create schema if not exists $VDK_TRINO_SCHEMA"

In [ ]:
!vdk trino-query -q "create table if not exists staging_area_trips (\
trip                       INTEGER ,\
date                       VARCHAR ,\
location                   VARCHAR ,\
tyres                      VARCHAR ,\
temperature_start_c        REAL ,\
temperature_end_c          REAL ,\
distance_km                INTEGER ,\
duration_minutes           INTEGER ,\
average_speed_kmh          INTEGER ,\
average_consumption_kwhkm  REAL ,\
charge_level_start         REAL ,\
charge_level_end           REAL ,\
ac_c                       VARCHAR ,\
heated_front_seats_level   INTEGER ,\
mode                       VARCHAR \
 )"

In [ ]:
! vdk ingest-csv -f "VW ID. 3 Pro Max EV Consumption.csv"  --table-name "staging_area_trips"

To verify that the data is ingested as expected, let's query the database. 

VDK comes with a easy way to query any pre-configured database using CLI: 




In [ ]:
! vdk trino-query -q "SELECT * FROM staging_area_trips"

<font color='blue'>**NOTICE:**</font> `vdk ingest-csv` might be great if you want to ingest a signle file quickly. But if we need continious ingestion over time of a lot more data we'd use Ingestion Data Jobs. For more information see [Ingestion examples](https://github.com/vmware/versatile-data-kit/wiki/Examples#ingestion-examples) 


### Step 2: Identify the Grain

At this point, we know the questions that we want to answer, we understand the business process and it’s now time to start prepping up our data warehouse design.

To be able to answer the request that we outlined above, we know that we will probably need the following attributes in our model.

* consumption per trip, duration for that trip (both in time and distance) , status 

And following descriptive data: 

* vehicle information - model, battery type, etc. 


Reminder: The grain is the business definition of what a single fact table record represents.  Thehe grain is the description of the measurement event in the physical world that gives rise to a measurement. Kimball proposes three types: Transactional, Periodic snapshot and Accumulating snapshot.

For example: When the grocery store scanner measures the quantity and the charged price of a product being purchased, the grain is literally the beep of the scanner. That is a great grain definition! 


<font color='red'>**ATTENTION!**</font> ...

**TASK: What is our grain definition for the above business request?**

<!-- 
In our case we probably can work with more aggregated data - like daily snapshot or weekly . But for the purposes of this tutorial we'd proceed with transactional data. The grain would be the end of the a full trip. 

So we seem we'd need a bit of  transactional data, (a single trip can be thought of as a transaction), 
We need also  descriptive data that goes along with these transactions (currently car temperature features but in the future all sorts of information that may impact battery life). This is a good indication that we may need both a fact and dimension.

To remind again, we can think of a fact table as a place where we can store measurements and a dimension table where we store descriptive attributes associated with the facts measurement.

**The grain of our data is at the transaction level** for a trip (we are going to track each trip). Possibly some time of aggregated level would do as well (snapshot grain). But this would give more flexibility to our ML model.
-->

### Step 3: Identify the dimensions

  Once the grain has been properly declared, the dimensions typically can easily be identified as they represent the “who, what, where, when, why, and how” associated with the event. A robust set of dimensions representing all possible descriptions should be identified. In our case we'd need data about the vehicle information (manufacturer, model, battery type etc.) 

<font color='red'>**ATTENTION!**</font> ...

**TASK: What dimensions are relevant to the above business request?**


### Step 4: Identify the facts

Facts are determined by answering the question, “What is the process measuring? In other words the facts are the metrics that business users are concerned about. These must be appropriately defined in accordance with the declared grain. If not, they should be placed in a different fact table. Usually, facts are numerical data, such as total cost or order quantity. 

<font color='red'>**ATTENTION!**</font> ...

**TASK: What is are fact table(s) relevant to the above business request?**

<!--

We have identified that our main fact table is "fact_trips" at transaction grain. -->

### Data transformation: Create a Data Job
Now that we have explored VDK's capabilities, and followed the dimensiton business process, let's create our transformation data jobs. 
 


In [ ]:
#!vdk create -n build-dimensional-model -t team-awesome -p /home/jovyan

When you create a data job, VDK automatically downloads some template scripts and files, so that you can get accustomed to the data job's structure. They are super helpful in getting you ready to run your own data jobs. However, let's go ahead and delete these for our example, since we won't be starting from scratch, but please check them out! Alternatively, you can explore the 'vdk create --no-template' option, if you do not want these templates downloaded. Let's go ahead and delete the following files:
* The SQL script: our example does not do anything with SQL.
* The sample Python script: we already have moved four sample Python scripts, so we won't be needing this.
* README.md: We already have a README for the entire example, so we can get rid of this.
* requirements.txt: Each data job would need this file if the data job relies on external libraries that VDK does not have. In our case, MyBinder installed those upon startup, so we won't be needing this either.

As such, please run the code below to delete them:

<font color='red'>**ATTENTION!**</font> Please change 'build-dimensional-model' to the name of your data job.

In [ ]:
! rm "build-dimensional-model/10_sql_step.sql"
! rm "build-dimensional-model/20_python_step.py"

Great! Now you're all set up with the data job:
* You have created a data job.
* You have deleted the template files that you do not need.
* You have moved the sample scripts we provided to the data job sub-folder.
* You have moved the raw CSV file to the data job sub-folder for a neater environment!

The next step is to begin working on each script in the data job! Let's do it!

We have 2 files (we can see them in samples/build-dimensional-model). Feel free to copy it.

* 10_fact_trips_create_table.sql - we define the schema of our fact table 
* 20_fact_trips_update.py - this will load the data into our fact table. 

The code looks something like this 
```
job_input.execute_template(
        template_name='periodic_snapshot',
        template_args={
            'source_view': 'staging_area_trips',
            'target_table': 'fact_trips',
            'last_arrival_ts': 'date'
        },
    )
```

We are using period_snapshot (called also append). <br>
Append strategy appends a snapshot of records observed between time t1 and t2 from the source table to the target table, truncating all present target table records observed after t1. <br>
The strategy can be used for updating Periodic Snapshot Fact Tables or transaction fact table in data warehousing ETL jobs. <br>
It suitable to update records where late arrival data is expected. 




In [ ]:
!vdk run samples/build-dimensional-model

In [ ]:
! vdk trino-query -q "select * from fact_trips"

<font color='blue'>**NOTICE:**</font> <br>
For other type of Kimball templates you can checkout [example documentation](https://github.com/vmware/versatile-data-kit/wiki/SQL-Data-Processing-templates-examples) . <br>
Template is pretty much a "reusable data job". Anyone that writes a data job, can create their own template. See [Tempalate registry documentation](https://github.com/vmware/versatile-data-kit/blob/main/projects/vdk-core/src/vdk/api/plugin/plugin_input.py#L87)



### Build the ML Model

It's time to build the model. We focus on build-ml-model data job

It has two steps. The first one would process the data so it is suitable for our ML model (linear regression)
The second will build the actual ML Model. 

After we are done building the model let's try to run our job

<font color='red'>**ATTENTION!**</font> Please change to the name of your data job if necessary

In [ ]:
! vdk run samples/build-ml-model

### Build a Streamlit Visualization

Now that we have finished with the data job, let's use that hard-earned model to make a cool dashboard!

When you run below command, you will get an output, but the kernel will be stuck. That's okay! Just open a new tab in your browser,
copy the link of the MyBinder environment, delete everything after "user/blah blah blah" and paste "/proxy/8501/"
So, something like this: 
```
https://hub.gke2.mybinder.org/user/alexanderavramo-n-example-empty-zkd8q00p/proxy/8501/
```

The Streamlit dashboard will now show up!



In [ ]:
! streamlit run samples/build_streamlit_dashboard.py

### Bonus: Deploy

Since the analysis that we perform is on a regular basis, it makes sense to schedule our data job to run once per week. VDK allows the **automatic execution of data jobs by deploying them on a cloud server** which handles the regular execution as per schedule that the user defines. The deployment configurations are entered in the **"config.ini"** file that is required for deployment.  
Let's open it up and examine the contents.

In the first section [owner], we have specified the **team owning the data job**. In the second section [job] we defined the schedule of execution. It is in cron format (you can use [this website](https://crontab.guru/#*/20_*_*_*_*) to translate the cron schedule into a human-readable form). In this case, we want the schedule to run on the Monday of each week at 00:01am US time. Since VDK uses UTC time for schedule execution, the cron schedule indicates 05:01am UTC time. 

The config file could also include a [contacts] section which specifies whether any **notifications** are sent to specific emails upon job execution success, failure or deployment. In our case, we have left those empty.

The last part of the config file contains the **VDK configuration settings** - the type of DB to which we will be ingesting, the DB location, schema and catalogue. 

For a full list and explanations of the configuration settings you could enter into the "config.ini" file of a data job, you can run the following command:

In [ ]:
!vdk config-help


Let's now deploy the data job. We would need to install vdk-server.  

The below commands are illusatrative and won't owrk in myBinder. See https://github.com/vmware/versatile-data-kit/wiki/Scheduling-a-Data-Job-for-automatic-execution for more information.

In [ ]:
!vdk deploy -n <job-name> -t team-awesome  -r "Initial deploy" -p /home/jovyan/<job-name>

In [ ]:
! vdk deploy --show -n <job-name> -t team-awesome


And if there's an issue revert: 

In [ ]:
! vdk deploy --update --job-version <old-version> -n ingest-<unique-suffix> -t team_awesome

**Please share your feedback** : https://bit.ly/vdk-dsc

